In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf

Importing the Dataset

In [ ]:
test_df = pd.read_csv('test.csv')
test_df.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [ ]:
train_df = pd.read_csv('train.csv')
train_df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [ ]:
train_df.shape

(159571, 8)

In [ ]:
train_df.loc[3,'comment_text']

'"\nMore\nI can\'t make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of ""types of accidents""  -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know.\n\nThere appears to be a backlog on articles for review so I guess there may be a delay until a reviewer turns up. It\'s listed in the relevant form eg Wikipedia:Good_article_nominations#Transport  "'

In [ ]:
train_df[(train_df['toxic']==0) & (train_df['severe_toxic']==0) & (train_df['obscene']==0) & (train_df['threat']==0) & 
          (train_df['insult']==0) & (train_df['identity_hate']==0)].count()

id               143346
comment_text     143346
toxic            143346
severe_toxic     143346
obscene          143346
threat           143346
insult           143346
identity_hate    143346
dtype: int64

In [ ]:
APPO = {
"aren't" : "are not",
"can't" : "cannot",
"couldn't" : "could not",
"didn't" : "did not",
"doesn't" : "does not",
"don't" : "do not",
"hadn't" : "had not",
"hasn't" : "has not",
"haven't" : "have not",
"he'd" : "he would",
"he'll" : "he will",
"he's" : "he is",
"i'd" : "I would",
"i'd" : "I had",
"i'll" : "I will",
"i'm" : "I am",
"isn't" : "is not",
"it's" : "it is",
"it'll":"it will",
"i've" : "I have",
"let's" : "let us",
"mightn't" : "might not",
"mustn't" : "must not",
"shan't" : "shall not",
"she'd" : "she would",
"she'll" : "she will",
"she's" : "she is",
"shouldn't" : "should not",
"that's" : "that is",
"there's" : "there is",
"they'd" : "they would",
"they'll" : "they will",
"they're" : "they are",
"they've" : "they have",
"we'd" : "we would",
"we're" : "we are",
"weren't" : "were not",
"we've" : "we have",
"what'll" : "what will",
"what're" : "what are",
"what's" : "what is",
"what've" : "what have",
"where's" : "where is",
"who'd" : "who would",
"who'll" : "who will",
"who're" : "who are",
"who's" : "who is",
"who've" : "who have",
"won't" : "will not",
"wouldn't" : "would not",
"you'd" : "you would",
"you'll" : "you will",
"you're" : "you are",
"you've" : "you have",
"'re": " are",
"wasn't": "was not",
"we'll":" will",
"didn't": "did not",
"tryin'":"trying"
}

In [ ]:
import re
def clean_text(text):
    
    text = text.lower()
    text = re.sub(r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', '', text) # clean url
    text = re.sub(r'#(\w+)', '', text)   # clean hashes
    text = re.sub(r'@(\w+)', '', text)   # clean @
    text = re.sub(r'<[^>]+>', '', text)  # clean tags
    text = re.sub(r'\d+', '', text)      # clean digits
    text = re.sub(r'[,!@\'\"?\.$%_&#*+-:;]', '', text)   # clean punctuation
    text = [APPO[word] if word in APPO else word for word in text.split()]  #
    
    return text

In [ ]:
train_df['comment_text'] = train_df['comment_text'].apply(clean_text)
test_df['comment_text'] = test_df['comment_text'].apply(clean_text)

In [ ]:
from sklearn.model_selection import train_test_split
train_set, val_set = train_test_split(train_df, test_size = 0.2, random_state=11)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words = 20000, oov_token='<oov>')
tokenizer.fit_on_texts(train_df.comment_text)

In [ ]:
traning_sequences = tokenizer.texts_to_sequences(train_set.comment_text)

In [ ]:
maxlen = max([len(x) for x in np.array(traning_sequences)])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
training_padded = pad_sequences(traning_sequences, maxlen = 500,
                                padding = 'pre',
                                truncating='pre')
train_y = np.array(train_set.loc[:,'toxic':])

In [ ]:
traning_sequences = tokenizer.texts_to_sequences(val_set.comment_text)
val_padded = pad_sequences(traning_sequences,maxlen = 500,
                                padding = 'pre',
                                truncating='pre')
val_y = np.array(val_set.loc[:,'toxic':])

In [ ]:
traning_sequences = tokenizer.texts_to_sequences(train_df.comment_text)
training_padded = pad_sequences(traning_sequences, maxlen = 500,
                                padding = 'pre',
                                truncating='pre')
train_y = np.array(train_df.loc[:,'toxic':])

In [ ]:
from keras.layers import Input, Dense, LSTM, Embedding, Dropout,GlobalMaxPooling1D
from keras.models import Model

inputLayer = Input(shape=500)
embedLayer = Embedding(input_dim=20000, output_dim=128)(inputLayer)
lstmLayer = LSTM(units=60, return_sequences=True)(embedLayer)
maxPool = GlobalMaxPooling1D()(lstmLayer)
dropOut1 = Dropout(0.1)(maxPool)
fcLayer1 = Dense(units=50, activation='relu')(dropOut1)
dropOut2 = Dropout(0.1)(fcLayer1)
fcLayer2 = Dense(units=6, activation='sigmoid')(dropOut2)

model = Model(inputs=inputLayer, outputs=fcLayer2)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(training_padded, train_y, batch_size=32, epochs=2)

Epoch 1/2
4987/4987 [==============================] - 1699s 337ms/step - loss: 0.1064 - accuracy: 0.8434
Epoch 2/2
4987/4987 [==============================] - 1694s 340ms/step - loss: 0.0453 - accuracy: 0.9847


In [ ]:
testing_sequences = tokenizer.texts_to_sequences(test_df.comment_text)
test_padded = pad_sequences(testing_sequences, maxlen = 500,
                                padding = 'pre',
                                truncating='pre')

In [ ]:
predicted = model.predict(test_padded, batch_size = 500)
predict = np.hstack((test_df.id[:, np.newaxis], predicted))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  


In [ ]:
subm = pd.DataFrame(predict, columns = ['id', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'])
subm.to_csv('subm.csv', index = False)

In [ ]:
subm.head(30)

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.997188,0.425211,0.963103,0.0808738,0.849878,0.160038
1,0000247867823ef7,0.000264645,1.28712e-07,4.38777e-05,1.15841e-06,1.00276e-05,8.77993e-06
2,00013b17ad220c46,0.0137195,2.48054e-05,0.00233132,0.000164896,0.00116399,0.000704795
3,00017563c3f7919a,0.000160038,4.53707e-08,1.66159e-05,3.51953e-07,4.50043e-06,5.95592e-06
4,00017695ad8997eb,0.00120032,9.91246e-07,0.000191242,7.71811e-06,6.1099e-05,4.6427e-05
5,0001ea8717f6de06,0.000424773,1.86434e-07,5.78309e-05,1.61712e-06,1.66623e-05,1.40776e-05
6,00024115d4cbde0f,0.000202745,5.46887e-08,2.06676e-05,3.90167e-07,5.81985e-06,6.85661e-06
7,000247e83dcc1211,0.571809,0.00906858,0.152236,0.0198196,0.142368,0.0384761
8,00025358d4737918,0.0821373,5.53705e-05,0.00393304,0.000299871,0.00877222,0.00455832
9,00026d1092fe71cc,0.000143707,6.77048e-08,1.91838e-05,4.50107e-07,4.344e-06,6.44575e-06
